<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/SwissLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/EngCorpus.pkl'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/tokenize_group_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/mask_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/knowledge_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/file_output_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/layers/dict_group_layer.py'
!wget 'https://raw.githubusercontent.com/IntelligentDDS/SwissLog/main/log_parser/offline_logparser/evaluator/evaluator.py'


In [3]:
import sys
sys.path.append('/content/') #to be able for importing libarary from this address

In [9]:
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541529 sha256=074ce7d50e52e50f17187b24b16bfea33eae7d46147c219dfc27b5238c233f5c
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja


In [11]:
from file_output_layer import FileOutputLayer
from knowledge_layer import KnowledgeGroupLayer
from mask_layer import MaskLayer
from tokenize_group_layer import TokenizeGroupLayer
from dict_group_layer import DictGroupLayer
import evaluator

import os
import re
import string
import hashlib
from datetime import datetime
from tqdm import tqdm
import pandas as pd

In [ ]:
input_dir = '../logs/' # The input directory of log file
output_dir = 'LogParserResult/' # The output directory of parsing results


def load_logs(log_file, regex, headers):
    """ Function to transform log file to dataframe
    """
    log_messages = dict()
    linecount = 0
    with open(log_file, 'r') as fin:
        for line in tqdm(fin.readlines(), desc='load data'):
            try:
                linecount += 1
                match = regex.search(line.strip())
                message = dict()
                for header in headers:
                    message[header] = match.group(header)
                message['LineId'] = linecount
                log_messages[linecount] = message
            except Exception as e:
                pass
    return log_messages

In [ ]:
def generate_logformat_regex(logformat):
    """ Function to generate regular expression to split log messages
    """
    headers = []
    splitters = re.split(r'(<[^<>]+>)', logformat)
    regex = ''
    for k in range(len(splitters)):
        if k % 2 == 0:
            splitter = re.sub(' +', '\\\s+', splitters[k])
            regex += splitter
        else:
            header = splitters[k].strip('<').strip('>')
            regex += '(?P<%s>.*?)' % header
            headers.append(header)
    regex = re.compile('^' + regex + '$')
    return headers, regex

benchmark_settings = {
    'HDFS': {
        'log_file': 'HDFS/HDFS_2k.log',
        'log_format': '<Date> <Time> <Pid> <Level> <Component>: <Content>',
        'regex': [r'blk_-?\d+', r'(\d+\.){3}\d+(:\d+)?']
        }
}

In [ ]:




if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--dictionary', default='../EngCorpus.pkl', type=str)
    args = parser.parse_args()
    corpus = args.dictionary

    benchmark_result = []
    for dataset, setting in benchmark_settings.items():
        print('\n=== Evaluation on %s ==='%dataset)
        indir = os.path.join(input_dir, os.path.dirname(setting['log_file']))
        outdir = os.path.join(output_dir, os.path.dirname(setting['log_file']))
        log_file = os.path.basename(setting['log_file'])

        filepath = os.path.join(indir, log_file)
        print('Parsing file: ' + filepath)
        starttime = datetime.now()
        headers, regex = generate_logformat_regex(setting['log_format'])
        log_messages = load_logs(filepath, regex, headers)
        # preprocess layer
        log_messages = KnowledgeGroupLayer(log_messages).run()
        # tokenize layer
        log_messages = TokenizeGroupLayer(log_messages, rex=setting['regex']).run()
        # dictionarize layer and cluster by wordset
        dict_group_result = DictGroupLayer(log_messages, corpus).run()
        # apply LCS and prefix tree
        results, templates = MaskLayer(dict_group_result).run()
        output_file = os.path.join(outdir, log_file)
        # output parsing results
        FileOutputLayer(log_messages, output_file, templates, ['LineId'] + headers).run()
        print('Parsing done. [Time taken: {!s}]'.format(datetime.now() - starttime))
        F1_measure, accuracy = evaluator.evaluate(
                            groundtruth=os.path.join(indir, log_file + '_structured.csv'),
                            parsedresult=os.path.join(outdir, log_file + '_structured.csv')
                            )
        benchmark_result.append([dataset, F1_measure, accuracy])

    print('\n=== Overall evaluation results ===')
    avg_accr = 0
    for i in range(len(benchmark_result)):
        avg_accr += benchmark_result[i][2]
    avg_accr /= len(benchmark_result)
    pd_result = pd.DataFrame(benchmark_result, columns={'dataset', 'F1_measure', 'Accuracy'})
    print(pd_result)
    print('avarage accuracy is {}'.format(avg_accr))
    pd_result.to_csv('benchmark_result.csv', index=False)
